<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/SA_PA_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import datetime
import sqlite3
import json

# --- Agent Configuration (Conceptual) ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash" # Your specified model name
    # In a real scenario, this would involve API keys and client setup
    # GOOGLE_API_KEY = "YOUR_GEMINI_API_KEY" # Placeholder

# --- 1. Conceptual Data Structures for Flights and Disruptions ---

class Flight:
    def __init__(self, flight_id, origin, destination, scheduled_departure, scheduled_arrival, aircraft_id, crew_id, gate_id, connection_pax=0, vip_pax=0, actual_departure=None, actual_arrival=None, status="On Schedule", disruptions=None, slack_minutes=0):
        self.flight_id = flight_id
        self.origin = origin
        self.destination = destination
        self.scheduled_departure = scheduled_departure
        self.scheduled_arrival = scheduled_arrival
        self.actual_departure = actual_departure if actual_departure is not None else scheduled_departure
        self.actual_arrival = actual_arrival if actual_arrival is not None else scheduled_arrival
        self.aircraft_id = aircraft_id
        self.crew_id = crew_id
        self.gate_id = gate_id
        self.connection_pax = connection_pax
        self.vip_pax = vip_pax
        self.status = status
        self.disruptions = disruptions if disruptions is not None else []
        self.slack_minutes = slack_minutes

    def __repr__(self):
        return (f"Flight {self.flight_id} ({self.aircraft_id}/{self.crew_id}/{self.gate_id}): {self.origin} to {self.destination} "
                f"(Sched: {self.scheduled_departure.strftime('%H:%M')} - {self.scheduled_arrival.strftime('%H:%M')}, "
                f"Actual: {self.actual_departure.strftime('%H:%M')} - {self.actual_arrival.strftime('%H:%M')}, "
                f"Status: {self.status}, Slack: {self.slack_minutes}min, Pax: C{self.connection_pax}/V{self.vip_pax})")

class DisruptionEvent:
    def __init__(self, event_id, description, affected_flight_id, start_time, duration_minutes, type, is_root_cause=False, caused_by_event_id=None, monetary_impact=0, affected_individuals=None):
        self.event_id = event_id
        self.description = description
        self.affected_flight_id = affected_flight_id
        self.start_time = start_time
        self.duration_minutes = duration_minutes
        self.type = type
        self.is_root_cause = is_root_cause
        self.caused_by_event_id = caused_by_event_id
        self.monetary_impact = monetary_impact
        self.affected_individuals = affected_individuals if affected_individuals is not None else {}

    def __repr__(self):
        root_status = " (ROOT CAUSE)" if self.is_root_cause else ""
        caused_by = f" (Caused by: {self.caused_by_event_id})" if self.caused_by_event_id else ""
        individuals_info = ""
        if self.affected_individuals:
            individuals_info = f" (Affected: VIP:{self.affected_individuals.get('VIP',0)}, Conn:{self.affected_individuals.get('Connecting',0)})"

        return (f"Disruption {self.event_id}{root_status}{caused_by}: {self.type} - {self.description} "
                f"on Flight {self.affected_flight_id} for {self.duration_minutes} mins."
                f" Monetary: ${self.monetary_impact:,.2f}{individuals_info}")

# --- Conceptual LLM Interaction ---
class ConceptualLLM:
    def __init__(self, model_name: str):
        self.model_name = model_name
        print(f"\n[LLM Concept] Initializing Conceptual LLM with model: {self.model_name}")
        # In a real scenario, this would involve `genai.configure` and `genai.GenerativeModel`
        # try:
        #     genai.configure(api_key=AgentConfig.GOOGLE_API_KEY)
        #     self.model = genai.GenerativeModel(model_name)
        #     print("[LLM Concept] Real Gemini API client conceptually initialized.")
        # except Exception as e:
        #     print(f"WARNING: Could not conceptually initialize Gemini API client: {e}. LLM calls will be simulated.")
        self.simulate_llm_response_counter = 0

    def analyze_disruption_impact_with_llm(self, disruption_details: dict, flight_details: dict, affected_flights_details: list[dict]) -> str:
        # This method would simulate calling the LLM to get a rich analysis
        self.simulate_llm_response_counter += 1
        prompt = f"""
        Analyze the following flight disruption and its potential ripple effects.
        Disruption: {disruption_details}
        Affected Flight Details: {flight_details}
        Ripple-Affected Flights Details: {affected_flights_details}

        Provide a concise summary of the critical impact areas, including resource constraints (aircraft, crew, gate),
        passenger impact (VIP, connecting), and potential operational challenges. Suggest immediate next steps for operations.
        Be thorough but brief.
        """

        # --- Simulate LLM's sophisticated analysis ---
        if self.simulate_llm_response_counter == 1:
            return f"""
            [LLM Analysis (Simulated for Root Cause {disruption_details.get('event_id', '')} using {self.model_name})]
            Root Cause: {disruption_details.get('description', '')} on {flight_details.get('flight_id', '')}.
            Impact Areas:
            - **Resource Gridlock**: Aircraft {flight_details.get('aircraft_id', '')} and Crew {flight_details.get('crew_id', '')} are significantly delayed, leading to probable misconnections for their next assigned flights. Gate {flight_details.get('gate_id', '')} is occupied.
            - **Passenger Stranding**: High count of connecting passengers ({flight_details.get('connection_pax',0)}) and VIPs ({flight_details.get('vip_pax',0)}) on the root flight will likely miss their connections.
            - **Operational Strain**: The delay propagates to {len(affected_flights_details)} subsequent flights, creating a cascading effect on crew duty times and aircraft utilization.
            Immediate Next Steps:
            1. Identify alternative aircraft for downstream rotations.
            2. Re-route/re-book connecting passengers and notify VIPs.
            3. Assess crew legality for subsequent flights; find backup crews.
            4. Coordinate with airport ops for new gate assignments or remote stands.
            """
        elif self.simulate_llm_response_counter == 2:
             return f"""
            [LLM Analysis (Simulated for Root Cause {disruption_details.get('event_id', '')} using {self.model_name})]
            Root Cause: {disruption_details.get('description', '')} on {flight_details.get('flight_id', '')}.
            Impact Areas:
            - **Gate Bottleneck**: Gate {flight_details.get('gate_id', '')} is unavailable, forcing flight {flight_details.get('flight_id', '')} to hold or divert. This impacts inbound and outbound operations at {flight_details.get('destination', '')}.
            - **Downstream Aircraft/Crew**: Aircraft {flight_details.get('aircraft_id', '')} and Crew {flight_details.get('crew_id', '')} will be delayed for their next leg if not swapped, affecting {len(affected_flights_details)} flights.
            - **Minimal Passenger Impact (Initial)**: Direct passenger impact might be lower initially, but connection impacts will grow if not managed.
            Immediate Next Steps:
            1. Secure alternative gate or remote parking for inbound.
            2. Evaluate possibility of aircraft/crew swap for subsequent legs.
            3. Proactively communicate with connecting passengers.
            """
        else:
             return f"""
             [LLM Analysis (Simulated General for Root Cause {disruption_details.get('event_id', '')} using {self.model_name})]
             Disruption Type: {disruption_details.get('type', '')}. The system identified a core issue affecting flight {disruption_details.get('affected_flight_id', '')}.
             This appears to be a systemic issue given its duration and propagation.
             Key affected resources include: {flight_details.get('aircraft_id', '')}, {flight_details.get('crew_id', '')}, and gate {flight_details.get('gate_id', '')}.
             Further investigation is recommended on previous legs impacting these resources.
             Consider proactive messaging to all potentially affected passengers beyond direct connections.
             """

# --- 2. Disruption Determination Control Unit (Conceptual Logic) ---

class DisruptionDeterminationControlUnit:
    def __init__(self, db_name=':memory:', disruption_threshold_minutes=15, use_llm=True):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.disruption_threshold_minutes = disruption_threshold_minutes
        self.last_event_id = 0
        self.demo_date = datetime.date(2025, 6, 30)

        self.aircraft_rotations = {}
        self.crew_rotations = {}
        self.resources_last_available = {
            "aircraft": {},
            "crew": {},
            "gate": {}
        }

        self._setup_database()
        self._load_initial_state_from_db()

        self.use_llm = use_llm
        self.llm_agent = ConceptualLLM(AgentConfig.LLM_MODEL_NAME) if use_llm else None

    def _setup_database(self):
        print(f"\n[DB Setup] Setting up SQLite database...")
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS flights (
                flight_id TEXT PRIMARY KEY,
                origin TEXT,
                destination TEXT,
                scheduled_departure TEXT,
                scheduled_arrival TEXT,
                actual_departure TEXT,
                actual_arrival TEXT,
                aircraft_id TEXT,
                crew_id TEXT,
                gate_id TEXT,
                connection_pax INTEGER,
                vip_pax INTEGER,
                status TEXT,
                disruptions TEXT,
                slack_minutes INTEGER
            )
        ''')
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS disruptions (
                event_id TEXT PRIMARY KEY,
                description TEXT,
                affected_flight_id TEXT,
                start_time TEXT,
                duration_minutes INTEGER,
                type TEXT,
                is_root_cause INTEGER,
                caused_by_event_id TEXT,
                monetary_impact REAL,
                affected_individuals TEXT
            )
        ''')
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS resources (
                resource_id TEXT PRIMARY KEY,
                resource_type TEXT,
                last_available TEXT
            )
        ''')
        self.conn.commit()
        print("[DB Setup] Database tables ensured.")

    def _save_flight(self, flight: Flight):
        disruptions_str = ",".join(flight.disruptions) if flight.disruptions else ""
        self.cursor.execute('''
            INSERT OR REPLACE INTO flights (flight_id, origin, destination, scheduled_departure, scheduled_arrival,
                                        actual_departure, actual_arrival, aircraft_id, crew_id, gate_id,
                                        connection_pax, vip_pax, status, disruptions, slack_minutes)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (flight.flight_id, flight.origin, flight.destination,
              flight.scheduled_departure.isoformat(), flight.scheduled_arrival.isoformat(),
              flight.actual_departure.isoformat(), flight.actual_arrival.isoformat(),
              flight.aircraft_id, flight.crew_id, flight.gate_id,
              flight.connection_pax, flight.vip_pax, flight.status, disruptions_str, flight.slack_minutes))
        self.conn.commit()

    def _get_flight(self, flight_id) -> Flight | None:
        self.cursor.execute("SELECT * FROM flights WHERE flight_id = ?", (flight_id,))
        row = self.cursor.fetchone()
        if row:
            (flight_id, origin, destination, sched_dep, sched_arr, actual_dep, actual_arr,
             aircraft_id, crew_id, gate_id, conn_pax, vip_pax, status, disruptions_str, slack_min) = row
            return Flight(
                flight_id, origin, destination,
                datetime.datetime.fromisoformat(sched_dep), datetime.datetime.fromisoformat(sched_arr),
                aircraft_id, crew_id, gate_id, conn_pax, vip_pax,
                actual_departure=datetime.datetime.fromisoformat(actual_dep),
                actual_arrival=datetime.datetime.fromisoformat(actual_arr),
                status=status,
                disruptions=disruptions_str.split(',') if disruptions_str else [],
                slack_minutes=slack_min
            )
        return None

    def _get_all_flights(self) -> list[Flight]:
        self.cursor.execute("SELECT flight_id FROM flights")
        return [self._get_flight(row[0]) for row in self.cursor.fetchall()]

    def _save_disruption_event(self, event: DisruptionEvent):
        affected_individuals_json = json.dumps(event.affected_individuals)
        self.cursor.execute('''
            INSERT OR REPLACE INTO disruptions (event_id, description, affected_flight_id, start_time, duration_minutes,
                                            type, is_root_cause, caused_by_event_id, monetary_impact, affected_individuals)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (event.event_id, event.description, event.affected_flight_id,
              event.start_time.isoformat(), event.duration_minutes, event.type,
              1 if event.is_root_cause else 0, event.caused_by_event_id,
              event.monetary_impact, affected_individuals_json))
        self.conn.commit()

    def _get_disruption_event(self, event_id) -> DisruptionEvent | None:
        self.cursor.execute("SELECT * FROM disruptions WHERE event_id = ?", (event_id,))
        row = self.cursor.fetchone()
        if row:
            (event_id, description, affected_flight_id, start_time, duration_minutes,
             type, is_root_cause_int, caused_by_event_id, monetary_impact, affected_individuals_json) = row
            return DisruptionEvent(
                event_id, description, affected_flight_id,
                datetime.datetime.fromisoformat(start_time), duration_minutes, type,
                is_root_cause=bool(is_root_cause_int), caused_by_event_id=caused_by_event_id,
                monetary_impact=monetary_impact, affected_individuals=json.loads(affected_individuals_json)
            )
        return None

    def _save_resource_state(self, resource_type, resource_id, last_available_time):
        unique_resource_id = f"{resource_type}_{resource_id}"
        self.cursor.execute('''
            INSERT OR REPLACE INTO resources (resource_id, resource_type, last_available)
            VALUES (?, ?, ?)
        ''', (unique_resource_id, resource_type, last_available_time.isoformat()))
        self.conn.commit()

    def _load_initial_state_from_db(self):
        self.cursor.execute("SELECT MAX(CAST(SUBSTR(event_id, 2) AS INTEGER)) FROM disruptions WHERE event_id LIKE 'E%'")
        max_event_id_from_disruptions = self.cursor.fetchone()[0]
        self.cursor.execute("SELECT MAX(CAST(SUBSTR(event_id, 2) AS INTEGER)) FROM disruptions WHERE event_id LIKE 'R%'")
        max_event_id_from_ripples = self.cursor.fetchone()[0]
        self.last_event_id = max(max_event_id_from_disruptions or 0, max_event_id_from_ripples or 0)

        self.cursor.execute("SELECT resource_type, resource_id, last_available FROM resources")
        for row in self.cursor.fetchall():
            resource_type, resource_id, last_available_str = row
            if resource_type not in self.resources_last_available:
                self.resources_last_available[resource_type] = {}
            self.resources_last_available[resource_type][resource_id] = datetime.datetime.fromisoformat(last_available_str)

        self.cursor.execute("SELECT flight_id, aircraft_id, crew_id, scheduled_departure FROM flights ORDER BY scheduled_departure")
        for flight_id, aircraft_id, crew_id, scheduled_departure_str in self.cursor.fetchall():
            self.aircraft_rotations.setdefault(aircraft_id, []).append(flight_id)
            self.crew_rotations.setdefault(crew_id, []).append(flight_id)

        for ac_id in self.aircraft_rotations:
            self.aircraft_rotations[ac_id].sort(key=lambda f_id: self._get_flight(f_id).scheduled_departure)
        for crew_id in self.crew_rotations:
            self.crew_rotations[crew_id].sort(key=lambda f_id: self._get_flight(f_id).scheduled_departure)

        print("\n[DB Load] Flights, disruptions, and resource states loaded from conceptual database.")


    def generate_flight_schedule(self, flight_data):
        print("\n--- Generating Initial Flight Schedule ---")
        for data in flight_data:
            dep_time = datetime.datetime.combine(self.demo_date, data['scheduled_departure'].time())
            arr_time = datetime.datetime.combine(self.demo_date, data['scheduled_arrival'].time())

            flight = Flight(data['flight_id'], data['origin'], data['destination'],
                            dep_time, arr_time,
                            data['aircraft_id'], data['crew_id'], data['gate_id'],
                            data.get('connection_pax',0), data.get('vip_pax',0))
            self._save_flight(flight)

            self.resources_last_available['aircraft'][flight.aircraft_id] = max(self.resources_last_available['aircraft'].get(flight.aircraft_id, datetime.datetime.combine(self.demo_date, datetime.time(0,0))), flight.scheduled_arrival)
            self.resources_last_available['crew'][flight.crew_id] = max(self.resources_last_available['crew'].get(flight.crew_id, datetime.datetime.combine(self.demo_date, datetime.time(0,0))), flight.scheduled_arrival)
            self.resources_last_available['gate'][flight.gate_id] = max(self.resources_last_available['gate'].get(flight.gate_id, datetime.datetime.combine(self.demo_date, datetime.time(0,0))), flight.scheduled_arrival)

            self._save_resource_state('aircraft', flight.aircraft_id, self.resources_last_available['aircraft'][flight.aircraft_id])
            self._save_resource_state('crew', flight.crew_id, self.resources_last_available['crew'][flight.crew_id])
            self._save_resource_state('gate', flight.gate_id, self.resources_last_available['gate'][flight.gate_id])


            self.aircraft_rotations.setdefault(flight.aircraft_id, []).append(flight.flight_id)
            self.crew_rotations.setdefault(flight.crew_id, []).append(flight.flight_id)

            print(f"Scheduled: {flight}")

        for ac_id in self.aircraft_rotations:
            self.aircraft_rotations[ac_id].sort(key=lambda f_id: self._get_flight(f_id).scheduled_departure)
        for crew_id in self.crew_rotations:
            self.crew_rotations[crew_id].sort(key=lambda f_id: self._get_flight(f_id).scheduled_departure)


    def receive_flight_schedule_update_inputs(self, update_inputs):
        print("\n--- Receiving Flight Schedule Update Inputs ---")
        new_disruptions_to_process = []
        for update in update_inputs:
            self.last_event_id += 1
            event_id = f"E{self.last_event_id}"
            disruption = DisruptionEvent(
                event_id=event_id,
                description=update['description'],
                affected_flight_id=update['affected_flight_id'],
                start_time=datetime.datetime.combine(self.demo_date, update['start_time'].time()),
                duration_minutes=update['duration_minutes'],
                type=update['type'],
                monetary_impact=update.get('monetary_impact', 0),
                affected_individuals=update.get('affected_individuals', {})
            )
            self._save_disruption_event(disruption)
            new_disruptions_to_process.append(disruption)
            print(f"Received update: {disruption.type} for {disruption.affected_flight_id}, {disruption.duration_minutes} min at {disruption.start_time.strftime('%H:%M')}")
        return new_disruptions_to_process

    def determine_disruptive_events(self, new_disruptions):
        print("\n--- Determining Disruptive Events ---")
        disruptive_events_identified = []
        for disruption_obj in new_disruptions:
            flight = self._get_flight(disruption_obj.affected_flight_id)
            if not flight:
                print(f"Warning: Flight {disruption_obj.affected_flight_id} not found for event {disruption_obj.event_id}.")
                continue

            initial_actual_departure = flight.actual_departure
            initial_actual_arrival = flight.actual_arrival

            flight.actual_arrival += datetime.timedelta(minutes=disruption_obj.duration_minutes)

            if disruption_obj.type in ["Departure Delay", "Crew Delay", "Maintenance Delay", "Gate Delay"]:
                flight.actual_departure += datetime.timedelta(minutes=disruption_obj.duration_minutes)
            elif disruption_obj.type in ["Arrival Delay", "Weather Delay", "Airport Congestion"]:
                turnaround_time_needed = datetime.timedelta(minutes=60)
                new_departure_after_turnaround = flight.actual_arrival + turnaround_time_needed
                if new_departure_after_turnaround > flight.actual_departure:
                     flight.actual_departure = new_departure_after_turnaround

            effective_delay = (flight.actual_departure - flight.scheduled_departure).total_seconds() / 60

            if effective_delay > flight.slack_minutes + self.disruption_threshold_minutes:
                disruption_obj.is_root_cause = True
                flight.status = "Disrupted"
                flight.disruptions.append(disruption_obj.event_id)
                disruptive_events_identified.append(disruption_obj)
                print(f"  > Event {disruption_obj.event_id} is DISRUPTIVE for {flight.flight_id} (Effective Delay: {effective_delay:.0f} min).")
                print(f"    New actual departure for {flight.flight_id}: {flight.actual_departure.strftime('%H:%M')}")

                self.resources_last_available['aircraft'][flight.aircraft_id] = flight.actual_arrival
                self.resources_last_available['crew'][flight.crew_id] = flight.actual_arrival
                self.resources_last_available['gate'][flight.gate_id] = flight.actual_arrival

                self._save_resource_state('aircraft', flight.aircraft_id, self.resources_last_available['aircraft'][flight.aircraft_id])
                self._save_resource_state('crew', flight.crew_id, self.resources_last_available['crew'][flight.crew_id])
                self._save_resource_state('gate', flight.gate_id, self.resources_last_available['gate'][flight.gate_id])

            else:
                print(f"  > Event {disruption_obj.event_id} for {flight.flight_id} is NOT disruptive (slack + threshold absorbed delay).")
                flight.status = "Delayed (absorbed)"
                if not (flight.actual_departure > initial_actual_departure or flight.actual_arrival > initial_actual_arrival):
                    flight.actual_departure = initial_actual_departure
                    flight.actual_arrival = initial_actual_arrival

            self._save_flight(flight)

        for d_event in new_disruptions:
            self._save_disruption_event(d_event)

        return disruptive_events_identified

    def analyze_ripple_effects_and_display_summary(self, root_disruptions):
        print("\n--- Analyzing Ripple Effects and Displaying Integrated Summary ---")
        for root_disruption_obj in root_disruptions:
            root_disruption = self._get_disruption_event(root_disruption_obj.event_id)
            print(f"\n--- Root Cause Identified (Conspicuous Indicia): {root_disruption} ---")
            root_flight = self._get_flight(root_disruption.affected_flight_id)
            print(f"  Root Flight Status: {root_flight.status}")

            affected_by_ripple_events = []

            # Process aircraft ripple effects
            aircraft_flights = self.aircraft_rotations.get(root_flight.aircraft_id, [])
            for f_id in aircraft_flights:
                if f_id == root_flight.flight_id:
                    continue
                if self._get_flight(f_id).scheduled_departure > root_flight.scheduled_departure:
                    flight = self._get_flight(f_id)
                    expected_aircraft_available = self.resources_last_available.get('aircraft', {}).get(root_flight.aircraft_id, root_flight.scheduled_arrival)
                    if flight.scheduled_departure < expected_aircraft_available:
                        delay_minutes = (expected_aircraft_available - flight.scheduled_departure).total_seconds() / 60
                        if delay_minutes > 0 and (delay_minutes > flight.slack_minutes + self.disruption_threshold_minutes / 2):
                            self.last_event_id += 1
                            ripple_event_id = f"R{self.last_event_id}"
                            ripple_disruption = DisruptionEvent(
                                event_id=ripple_event_id,
                                description=f"Aircraft {root_flight.aircraft_id} rotation delay from {root_flight.flight_id}",
                                affected_flight_id=flight.flight_id,
                                start_time=flight.scheduled_departure,
                                duration_minutes=int(delay_minutes),
                                type="Aircraft Rotation Ripple",
                                monetary_impact=25000,
                                affected_individuals={'VIP': int(flight.vip_pax * 0.5), 'Connecting': flight.connection_pax}
                            )
                            self._save_disruption_event(ripple_disruption)
                            flight.status = "Disrupted by Ripple"
                            flight.actual_departure += datetime.timedelta(minutes=ripple_disruption.duration_minutes)
                            flight.actual_arrival += datetime.timedelta(minutes=ripple_disruption.duration_minutes)
                            flight.disruptions.append(ripple_disruption.event_id)
                            self._save_flight(flight)
                            affected_by_ripple_events.append(ripple_disruption)
                            self.resources_last_available['aircraft'][flight.aircraft_id] = flight.actual_arrival
                            self._save_resource_state('aircraft', flight.aircraft_id, self.resources_last_available['aircraft'][flight.aircraft_id])


            # Process crew ripple effects
            crew_flights = self.crew_rotations.get(root_flight.crew_id, [])
            for f_id in crew_flights:
                if f_id == root_flight.flight_id:
                    continue
                flight = self._get_flight(f_id)
                if flight.scheduled_departure > root_flight.scheduled_departure:
                    expected_crew_available = self.resources_last_available.get('crew', {}).get(root_flight.crew_id, flight.scheduled_departure)
                    if flight.scheduled_departure < expected_crew_available:
                        delay_minutes = (expected_crew_available - flight.scheduled_departure).total_seconds() / 60
                        if delay_minutes > 0 and (delay_minutes > flight.slack_minutes + self.disruption_threshold_minutes / 2):
                            self.last_event_id += 1
                            ripple_event_id = f"R{self.last_event_id}"
                            ripple_disruption = DisruptionEvent(
                                event_id=ripple_event_id,
                                description=f"Crew {root_flight.crew_id} duty time violation from {root_flight.flight_id}",
                                affected_flight_id=flight.flight_id,
                                start_time=flight.scheduled_departure,
                                duration_minutes=int(delay_minutes),
                                type="Crew Scheduling Ripple",
                                monetary_impact=30000,
                                affected_individuals={'VIP': int(flight.vip_pax * 0.7), 'Connecting': flight.connection_pax}
                            )
                            self._save_disruption_event(ripple_disruption)
                            flight.status = "Disrupted by Ripple"
                            flight.actual_departure += datetime.timedelta(minutes=ripple_disruption.duration_minutes)
                            flight.actual_arrival += datetime.timedelta(minutes=ripple_disruption.duration_minutes)
                            flight.disruptions.append(ripple_disruption.event_id)
                            self._save_flight(flight)
                            affected_by_ripple_events.append(ripple_disruption)
                            self.resources_last_available['crew'][flight.crew_id] = flight.actual_arrival
                            self._save_resource_state('crew', flight.crew_id, self.resources_last_available['crew'][flight.crew_id])


            # Display affected disruptions (Caused-Disruption Indicia)
            if affected_by_ripple_events:
                print("\n  Other Disruptions Caused by this Root Event (Caused-Disruption Indicia):")
                for affected_disruption in affected_by_ripple_events:
                    db_disruption = self._get_disruption_event(affected_disruption.event_id)
                    db_disruption.caused_by_event_id = root_disruption.event_id
                    self._save_disruption_event(db_disruption)
                    print(f"    - {db_disruption}")

            # Display Monetary Impact (Monetary Impact Indicator) and Affected Individuals
            if root_disruption.monetary_impact == 0:
                root_disruption.monetary_impact = 100000 * (root_disruption.duration_minutes / 60) + \
                                                   root_flight.connection_pax * 50 + root_flight.vip_pax * 200
            self._save_disruption_event(root_disruption)

            print(f"\n--- Indicating Root Cause of Disruptions (Monetary Impact & Affected Individuals) ---")
            print(f"  Total Monetary Impact for Root Cause {root_disruption.event_id}: ${root_disruption.monetary_impact:,.2f}")
            if root_disruption.affected_individuals:
                print(f"  Root Cause Affected Individuals: VIP: {root_disruption.affected_individuals.get('VIP',0)}, Connecting: {root_disruption.affected_individuals.get('Connecting',0)}")

            # --- LLM Integration Point: Generate detailed analysis and recommendations ---
            if self.llm_agent:
                print("\n--- LLM-Powered Insight & Recommendations ---")
                root_disruption_dict = {k: str(v) if isinstance(v, (datetime.datetime, datetime.date)) else v for k, v in root_disruption.__dict__.items()}
                flight_details_dict = {k: str(v) if isinstance(v, (datetime.datetime, datetime.date)) else v for k, v in root_flight.__dict__.items()}
                affected_flights_details_list = []
                for ripple_event in affected_by_ripple_events:
                    ripple_flight = self._get_flight(ripple_event.affected_flight_id)
                    if ripple_flight:
                        affected_flights_details_list.append({k: str(v) if isinstance(v, (datetime.datetime, datetime.date)) else v for k, v in ripple_flight.__dict__.items()})

                llm_analysis = self.llm_agent.analyze_disruption_impact_with_llm(
                    root_disruption_dict,
                    flight_details_dict,
                    affected_flights_details_list
                )
                print(llm_analysis)

            print("\n  Current Overall Flight Schedule Status (from DB):")
            all_flights = self._get_all_flights()
            for flight in all_flights:
                if flight.status == "Disrupted" or flight.status == "Disrupted by Ripple" or flight.slack_minutes > 0:
                     print(f"  {flight} (Disruption Event IDs: {', '.join(flight.disruptions) if flight.disruptions else 'None'})")

# --- 3. Mocked Data and Demo Execution ---

if __name__ == "__main__":
    # Use a file-based SQLite database for persistence in a conceptual way
    db_file_name = 'flight_data.db'
    control_unit = DisruptionDeterminationControlUnit(db_name=db_file_name, disruption_threshold_minutes=15, use_llm=True)

    # Optional: Clear database for a fresh run (simulate starting fresh)
    try:
        conn = sqlite3.connect(db_file_name)
        cursor = conn.cursor()
        cursor.execute("DROP TABLE IF EXISTS flights")
        cursor.execute("DROP TABLE IF EXISTS disruptions")
        cursor.execute("DROP TABLE IF EXISTS resources")
        conn.commit()
        conn.close()
        print(f"\n[DB Reset] Previous database '{db_file_name}' cleared for fresh demo run.")
        control_unit = DisruptionDeterminationControlUnit(db_name=db_file_name, disruption_threshold_minutes=15, use_llm=True)
    except Exception as e:
        print(f"Could not clear database (it might not exist yet or be locked): {e}")


    # --- Mocked Initial Hypothetical Flight Schedule Data with 10 Flights ---
    # The patent refers to flights having origins, destinations, and scheduled times.
    # It also mentions aircraft, crew, and gate resources.
    initial_flight_data = [
        {'flight_id': 'DL100', 'origin': 'ATL', 'destination': 'LGA', # Early morning hub flight
         'scheduled_departure': datetime.datetime(2025, 6, 30, 6, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 8, 0),
         'aircraft_id': 'N100DL', 'crew_id': 'C01', 'gate_id': 'A10', 'connection_pax': 70, 'vip_pax': 8},
        {'flight_id': 'DL101', 'origin': 'LGA', 'destination': 'ORD', # Connects from DL100 Aircraft/Crew
         'scheduled_departure': datetime.datetime(2025, 6, 30, 9, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 10, 30),
         'aircraft_id': 'N100DL', 'crew_id': 'C01', 'gate_id': 'A10', 'connection_pax': 50, 'vip_pax': 5},
        {'flight_id': 'DL200', 'origin': 'ATL', 'destination': 'MIA', # Another ATL departure
         'scheduled_departure': datetime.datetime(2025, 6, 30, 7, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 9, 0),
         'aircraft_id': 'N200DL', 'crew_id': 'C02', 'gate_id': 'B20', 'connection_pax': 60, 'vip_pax': 7},
        {'flight_id': 'DL201', 'origin': 'MIA', 'destination': 'DFW', # Connects from DL200 Aircraft/Crew
         'scheduled_departure': datetime.datetime(2025, 6, 30, 10, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 12, 0),
         'aircraft_id': 'N200DL', 'crew_id': 'C02', 'gate_id': 'B20', 'connection_pax': 40, 'vip_pax': 4},
        {'flight_id': 'DL300', 'origin': 'LAX', 'destination': 'JFK', # Cross-country red-eye type
         'scheduled_departure': datetime.datetime(2025, 6, 30, 8, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 16, 0),
         'aircraft_id': 'N300DL', 'crew_id': 'C03', 'gate_id': 'C30', 'connection_pax': 90, 'vip_pax': 12},
        {'flight_id': 'DL301', 'origin': 'JFK', 'destination': 'FLL', # Connects from DL300 Aircraft/Crew
         'scheduled_departure': datetime.datetime(2025, 6, 30, 17, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 20, 0),
         'aircraft_id': 'N300DL', 'crew_id': 'C03', 'gate_id': 'C30', 'connection_pax': 80, 'vip_pax': 10},
        {'flight_id': 'DL400', 'origin': 'ORD', 'destination': 'DEN', # Mid-morning independent
         'scheduled_departure': datetime.datetime(2025, 6, 30, 10, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 12, 0),
         'aircraft_id': 'N400DL', 'crew_id': 'C04', 'gate_id': 'D40', 'connection_pax': 30, 'vip_pax': 3},
        {'flight_id': 'DL500', 'origin': 'DFW', 'destination': 'PHX', # Another independent
         'scheduled_departure': datetime.datetime(2025, 6, 30, 11, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 12, 30),
         'aircraft_id': 'N500DL', 'crew_id': 'C05', 'gate_id': 'E50', 'connection_pax': 25, 'vip_pax': 2},
        {'flight_id': 'DL600', 'origin': 'SEA', 'destination': 'SFO', # West coast short-haul
         'scheduled_departure': datetime.datetime(2025, 6, 30, 12, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 13, 0),
         'aircraft_id': 'N600DL', 'crew_id': 'C06', 'gate_id': 'F60', 'connection_pax': 15, 'vip_pax': 1},
        {'flight_id': 'DL700', 'origin': 'MCO', 'destination': 'CLT', # Afternoon independent
         'scheduled_departure': datetime.datetime(2025, 6, 30, 13, 0),
         'scheduled_arrival': datetime.datetime(2025, 6, 30, 14, 30),
         'aircraft_id': 'N700DL', 'crew_id': 'C07', 'gate_id': 'G70', 'connection_pax': 20, 'vip_pax': 2},
    ]
    control_unit.generate_flight_schedule(initial_flight_data)

    # --- Scenario 1: Minor Maintenance Delay on DL100 (Non-Disruptive, absorbed by ample slack) ---
    print("\n\n===== Scenario 1: Minor Maintenance Delay (Non-Disruptive) =====")
    flight_dl100 = control_unit._get_flight('DL100')
    flight_dl100.slack_minutes = 40 # Simulating ample schedule slack for turnaround at LGA
    control_unit._save_flight(flight_dl100)

    update_input_1 = [
        {'description': 'Routine post-flight check delay', 'affected_flight_id': 'DL100',
         'start_time': datetime.datetime(2025, 6, 30, 8, 5), 'duration_minutes': 20,
         'type': 'Maintenance Delay', 'affected_individuals': {'VIP': 0, 'Connecting': 0}}
    ]
    new_disruptions_1 = control_unit.receive_flight_schedule_update_inputs(update_input_1)
    disruptive_events_1 = control_unit.determine_disruptive_events(new_disruptions_1)

    if not disruptive_events_1:
        print("\nDisplay (Conceptual): No disruptions noted (delay absorbed by slack).")
    else:
        control_unit.analyze_ripple_effects_and_display_summary(disruptive_events_1)

    # --- Scenario 2: Significant Weather Delay on DL300 (Root Cause with multiple Ripple Effects) ---
    print("\n\n===== Scenario 2: Significant Weather Delay (Root Cause with Ripple Effect) =====")
    flight_dl300 = control_unit._get_flight('DL300')
    flight_dl300.slack_minutes = 15 # Simulating very limited slack at JFK
    control_unit._save_flight(flight_dl300)

    update_input_2 = [
        {'description': 'Severe thunderstorms at LAX affecting departure window', 'affected_flight_id': 'DL300',
         'start_time': datetime.datetime(2025, 6, 30, 7, 45), 'duration_minutes': 120, # 2-hour delay
         'type': 'Weather Delay', 'monetary_impact': 750000,
         'affected_individuals': {'VIP': flight_dl300.vip_pax, 'Connecting': flight_dl300.connection_pax}}
    ]
    new_disruptions_2 = control_unit.receive_flight_schedule_update_inputs(update_input_2)
    disruptive_events_2 = control_unit.determine_disruptive_events(new_disruptions_2)

    if disruptive_events_2:
        control_unit.analyze_ripple_effects_and_display_summary(disruptive_events_2)
    else:
        print("\nDisplay (Conceptual): No disruptive events identified in Scenario 2.")

    # --- Scenario 3: Gate Availability Issue on DL201 (Root Cause affecting multiple resources) ---
    print("\n\n===== Scenario 3: Gate Availability Issue (New Root Cause) =====")
    flight_dl201 = control_unit._get_flight('DL201')
    flight_dl201.slack_minutes = 5 # Tight turnaround
    control_unit._save_flight(flight_dl201)

    # Simulate gate B20 being occupied longer than expected
    control_unit.resources_last_available['gate']['B20'] = datetime.datetime(2025, 6, 30, 9, 45) # Gate became available later than DL201 needs it
    control_unit._save_resource_state('gate', 'B20', control_unit.resources_last_available['gate']['B20'])


    update_input_3 = [
        {'description': 'Gate B20 occupied by inbound aircraft', 'affected_flight_id': 'DL201',
         'start_time': datetime.datetime(2025, 6, 30, 9, 30), 'duration_minutes': 60, # 60 min gate delay
         'type': 'Gate Delay', 'monetary_impact': 150000,
         'affected_individuals': {'VIP': 0, 'Connecting': 0}} # Direct passengers not affected unless it ripples
    ]
    new_disruptions_3 = control_unit.receive_flight_schedule_update_inputs(update_input_3)
    disruptive_events_3 = control_unit.determine_disruptive_events(new_disruptions_3)

    if disruptive_events_3:
        control_unit.analyze_ripple_effects_and_display_summary(disruptive_events_3)
    else:
        print("\nDisplay (Conceptual): No disruptive events identified in Scenario 3.")

    # Close the database connection at the end of the conceptual demo
    control_unit.conn.close()
    print(f"\n[DB Operation] Conceptual database '{db_file_name}' connection closed.")


[DB Setup] Setting up SQLite database...
[DB Setup] Database tables ensured.

[DB Load] Flights, disruptions, and resource states loaded from conceptual database.

[LLM Concept] Initializing Conceptual LLM with model: gemini-2.5-flash

[DB Reset] Previous database 'flight_data.db' cleared for fresh demo run.

[DB Setup] Setting up SQLite database...
[DB Setup] Database tables ensured.

[DB Load] Flights, disruptions, and resource states loaded from conceptual database.

[LLM Concept] Initializing Conceptual LLM with model: gemini-2.5-flash

--- Generating Initial Flight Schedule ---
Scheduled: Flight DL100 (N100DL/C01/A10): ATL to LGA (Sched: 06:00 - 08:00, Actual: 06:00 - 08:00, Status: On Schedule, Slack: 0min, Pax: C70/V8)
Scheduled: Flight DL101 (N100DL/C01/A10): LGA to ORD (Sched: 09:00 - 10:30, Actual: 09:00 - 10:30, Status: On Schedule, Slack: 0min, Pax: C50/V5)
Scheduled: Flight DL200 (N200DL/C02/B20): ATL to MIA (Sched: 07:00 - 09:00, Actual: 07:00 - 09:00, Status: On Schedul